<a href="https://colab.research.google.com/github/kaustubh-2187/CogniAble/blob/master/Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/kaustubh-2187/CogniAble

In [ ]:
import os
HOME = os.getcwd()
HOME

# 1. Emotion Detection using MTCNN and ViT

In [57]:
HOME = os.path.join(HOME,"drive","MyDrive","Submissions","CogniAble")
%cd {HOME}

/content/drive/MyDrive/Submissions/CogniAble


In [30]:
import cv2
import numpy as np
import math
import matplotlib.pyplot as plt
from transformers import AutoImageProcessor, AutoModelForImageClassification
import torch
from mtcnn.mtcnn import MTCNN
import tensorflow as tf

In [35]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [38]:
def main(source_path, destination_path):

    # Huggingface Model for Emotion Detection
    processor = AutoImageProcessor.from_pretrained("trpakov/vit-face-expression")
    model = AutoModelForImageClassification.from_pretrained("trpakov/vit-face-expression")

    # Move the model to the chosen device (CPU or GPU)
    model.to(device)

    # Emotion labels
    emotion_labels = {
        0: 'Angry',
        1: 'Disgust',
        2: 'Fear',
        3: 'Happy',
        4: 'Neutral',
        5: 'Sad',
        6: 'Surprise'
    }

    # Create MTCNN detector for face detection
    mtcnn_detector = MTCNN()

    # Load the video capture object
    cap = cv2.VideoCapture(source_path)

    # Define video writer parameters (adjust as needed)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Video codec for mp4 format
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    out_video = cv2.VideoWriter(destination_path, fourcc, 20.0, (frame_width, frame_height))

    while True:
      # Capture frame-by-frame
      ret, frame = cap.read()

      # Exit if no frame is captured
      if not ret:
        break

      # Convert frame to RGB format (MTCNN expects RGB)
      rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

      # Detect faces using MTCNN
      faces = mtcnn_detector.detect_faces(rgb_frame)

      # Loop through detected faces
      for face in faces:
        # Extract bounding box coordinates
        x, y, w, h = face['box']

        # Extract the face region of interest (ROI)
        face_roi = frame[y:y+h, x:x+w]

        # Preprocess the face ROI for the ViT model
        preprocessed_face = processor(face_roi, return_tensors="pt")

        # Move the preprocessed data to the chosen device (CPU or GPU)
        preprocessed_face_gpu = preprocessed_face.to(device)

        # Use the ViT model to predict emotions on the preprocessed face (on GPU)
        with torch.no_grad():
          outputs = model(**preprocessed_face_gpu)
          predictions = torch.argmax(outputs.logits, dim=-1)

        # Extract the predicted emotion label
        predicted_emotion_label = predictions.item()
        predicted_emotion = emotion_labels[predicted_emotion_label]

        # Display the emotion on the frame
        cv2.putText(frame, predicted_emotion, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

        # Draw a rectangle around the face
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

      # Write the processed frame to the output video
      out_video.write(frame)

## 2. Video File Paths

Set the value of the variable `source_path` as the path to the video file that needs to be analyzed

Set the value of the variable `destination_path` as the path where the transformed video will be stored

In [ ]:
source_path = './video.mp4'
destination_path = './gaze-transformer/output.mp4'
main(source_path, destination_path)

1/1 [==============================] - 0s 53ms/step


# 2. Gaze Transformer

**IMPORATANT NOTE** : Download the weights for the Gaze Transformer Model from
[here](https://tumde-my.sharepoint.com/:f:/g/personal/zhifan_ni_tum_de/Ev6sVnE0y2VBnmJ4RD65W7EB7PVDuGQ68Ybkaj31dGCUow?e=t29y5C)

Once downloaded, move all the downloaded folders to the `weights` directory in the `gaze-transformer` directory

In [43]:
HOME

'/content/drive/MyDrive/Submissions/CogniAble'

In [45]:
gaze_transformer_path = os.path.join(HOME,"gaze-transformer")
%cd {gaze_transformer_path}

/content/drive/MyDrive/Submissions/CogniAble/gaze-transformer


In [46]:
!mkdir output_video_directory
!cd output_video_directory
!mkdir video

In [ ]:
!pip install torch==1.13.0 torchvision torchaudio torchtext==0.14.0 torchdata==0.5.0
!pip install numpy opencv-python Pillow PyYAML requests scipy tqdm tensorboard wandb pandas seaborn Cython easydict gdown scikit-learn imageio notebook ipywidgets black

In [47]:
!python run.py --source ./output.mp4 --out ./output_video_directory/ --future 3 --hoi-thres 0.3 --print

Traceback (most recent call last):
  File "/content/drive/MyDrive/Submissions/CogniAble/gaze-transformer/run.py", line 19, in <module>
    from common.transforms import STTranTransform, YOLOv5Transform
  File "/content/drive/MyDrive/Submissions/CogniAble/gaze-transformer/common/transforms.py", line 7, in <module>
    from modules.object_tracking.yolov5.utils.augmentations import letterbox
  File "/content/drive/MyDrive/Submissions/CogniAble/gaze-transformer/modules/object_tracking/__init__.py", line 4, in <module>
    from .feature_backbone import FeatureExtractionResNet101, FeatureExtractionResNet50
  File "/content/drive/MyDrive/Submissions/CogniAble/gaze-transformer/modules/object_tracking/feature_backbone.py", line 7, in <module>
    from torchvision.models.resnet import ResNet, Bottleneck, model_urls
ImportError: cannot import name 'model_urls' from 'torchvision.models.resnet' (/usr/local/lib/python3.10/dist-packages/torchvision/models/resnet.py)
